In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
df = pd.read_csv("../../output/current_values.csv", index_col=0)


hf = pd.read_csv("../../output/Codes/2digit_occupations.csv", index_col=0)
hf = hf.set_index("codes")
cf = pd.read_csv("../../output/Codes/1digit_occupations.csv", index_col=0)
cf = cf.set_index("codes")


In [ ]:
import numpy as np
import pandas as pd
from pandas._libs.lib import is_integer

def weighted_qcut(values, weights, q, **kwargs):
    'Return weighted quantile cuts from a given series, values.'
    if is_integer(q):
        quantiles = np.linspace(0, 1, q + 1)
    else:
        quantiles = q
    order = weights.iloc[values.argsort()].cumsum()
    bins = pd.cut(order / order.iloc[-1], quantiles, **kwargs)
    return bins.sort_index()

sf['weighted_sp'] = pd.qcut(sf["log_1980"], 100, labels=False)




In [ ]:
p={}
u=[]
grouped = df.groupby(["Year","3digit occupation"])
for x in df["3digit occupation"].sort_values().unique():
    try:
        g = grouped.get_group((1980,x))
    except KeyError:
        continue
    l = g["Asecwt"].sum()
    g['weighted_wage']= g["Wage"]*g["Asecwt"]/l
    p[x] = g["weighted_wage"].sum()
    u.append(l)
i = pd.DataFrame({"occ":list(p.keys()), "mean_wage":list(p.values()), "asecwt":u})


In [ ]:
#df = df.join(kf, lsuffix='_caller', rsuffix='_other')
def graph(frame):
    arrays=[]
    sf = pd.DataFrame()
    grouped = frame.groupby('3digit occupation')
    for x in i["occ"].unique():
        try:
            arrays.append(grouped.get_group((x))[["3digit occupation","Wage", "Asecwt"]].to_numpy())
        except:
            continue
            
    for array in arrays:
        p = array[0,0]
        values = array[:,1]
        weights = array[:,2]
        loggy = np.log(np.average(values,weights=weights))
        mean = np.average(values,weights=weights)
        sf = sf.append({"occ":p,"loggy":loggy, "mean_wage":mean},ignore_index=True)
    
    return sf
kf = df[df["Year"]==1980]    
hf = df[df["Year"]==2010] 
sf = graph(kf)
sf = sf.rename({"loggy":"log_1980"},axis=1)
lf = graph(hf)
lf = lf.rename({"loggy":"log_2005"},axis=1)

In [ ]:
v = pd.merge(sf,lf,on="occ")
v["log"]=v["log_2005"]-v["log_1980"]

#v = pd.merge(v,i,on="occ")
v = v.drop(["occ","mean_wage_x","mean_wage_y","log_1980","log_2005"],axis=1)
#v = v.groupby("weighted_sp")["log"].mean().reset_index()
t = v.sort_values(by="weighted_sp")
v = v.sort_values(by="weighted_sp").to_numpy()
#v[v["log"]<0]
t

In [ ]:
plt.plot(v[:,0],v[:,-1])
